In [ ]:
%cd ..

In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
from pprint import pprint
from glob import glob
import pkg_resources

from pymedext_eds.annotators import Endlines, SentenceTokenizer, Hypothesis, \
                                    ATCDFamille, SyntagmeTokenizer, Negation, RegexMatcher, \
                                    QuickUMLSAnnotator, Pipeline
from pymedext_eds.utils import rawtext_loader
from pymedext_eds.viz import display_annotations

In [3]:
data_path = pkg_resources.resource_filename('pymedext_eds', 'data/demo')
file_list = glob(data_path + '/*.txt')
chunk = [rawtext_loader(x) for x in file_list]

In [4]:
endlines = Endlines(['raw_text'], 'endlines', 'endlines:v1')
sentences = SentenceTokenizer(['endlines'], 'sentence', 'sentenceTokenizer:v1')
hypothesis = Hypothesis(['sentence'], 'hypothesis', 'hypothesis:v1')
family = ATCDFamille(['sentence'], 'context', 'ATCDfamily:v1')
syntagmes = SyntagmeTokenizer(['sentence'], 'syntagme', 'SyntagmeTokenizer:v1')
negation = Negation(['syntagme'], 'negation', 'Negation:v1')
regex = RegexMatcher(['endlines','syntagme'], 'regex', 'RegexMatcher:v1', 'list_regexp.json')
umls = QuickUMLSAnnotator(['syntagme'], 'umls', 'QuickUMLS:2020AA', 
                          quickumls_fp='/Users/antoine/git/QuickUMLS/umls_data/',
                            overlapping_criteria='length',
                            threshold=0.9,
                            similarity_name='jaccard',
                            window=5)

pipeline = Pipeline(pipeline = [endlines, sentences, hypothesis, family, syntagmes, negation, regex, umls])

In [5]:
annotated_chunk = pipeline.annotate(chunk)

/Users/antoine/git/pymedext_eds/pymedext_eds/annotators.py:548: FutureWarning: Possible nested set at position 43
  for m in re.finditer(rex['regexp'], syntagme.value, flags=reflags):


In [13]:
pprint(chunk[0].get_annotations('regex')[10].to_dict())

{'ID': '8a5d9084-2d9d-11eb-bd3f-3c7d0a00025d',
 'attributes': {'context': 'patient',
                'hypothesis': 'certain',
                'id_regexp': 'id_regexp_CSE200048_cardiovascular',
                'label': 'CSE200048 Cardiovascular',
                'negation': 'aff',
                'snippet': 'ion au décours de laquelle était survenue une '
                           'phlébite avec embolie pulmonaire ,en 1974 , . Elle '
                           'avait été traitée par du PROZA',
                'version': 'v1'},
 'isEntity': True,
 'ngram': None,
 'source': 'RegexMatcher:v1',
 'source_ID': '8a59e380-2d9d-11eb-bd3f-3c7d0a00025d',
 'span': (8623, 8630),
 'type': 'regex',
 'value': 'embolie'}


In [7]:
display_annotations(chunk[0], ['umls'], attributes = ['negation', 'hypothesis','context', 'semtypes'], label_key = 'cui')